In [1]:
import numpy as np

from orc.data_structures import *
from orc.branch import *
from orc.utils import generate_problem
from orc.primal import *
from orc.relaxation import *
from orc.reduction import *
from orc.callbacks import *

In [2]:
A, b = generate_problem(10, 20)

In [3]:
bb = BranchAndBound(branch_strategy=branch_reduced_costs, 
                    lb_strategy=subgrad_opt,
                    callbacks=[PrimalHeurCallback(),
                               LagrPenaltiesReductionCallback(),
                               ColumnInclusionCallback()
                               ]
                    )

In [4]:
bb.search(A, b)

In [5]:
print(bb.best)

Node(level=-1, x0=[ 0  1  2  5  7  8  9 13 14 16 17 18 19], x1=[ 3  4  6 10 11 12 15], val=2475.0, lb=None, x_lb=None, lambd=None)


In [6]:
bb.best.x1, str(bb.best)

(array([ 3,  4,  6, 10, 11, 12, 15], dtype=int64),
 'Node(level=-1, x0=[ 0  1  2  5  7  8  9 13 14 16 17 18 19], x1=[ 3  4  6 10 11 12 15], val=2475.0, lb=None, x_lb=None, lambd=None)')

In [7]:
x = np.zeros(A.shape[-1])
x[bb.best.x1] = 1
x

array([0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0.,
       0., 0., 0.])

In [8]:
A @ x >= b

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [9]:
np.sum(A, axis=0) @ x

2475.0

In [ ]:
bb.node_count

In [ ]:
2 ** 20

In [ ]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

m = gp.Model("mip")
x = m.addMVar(A.shape[-1], vtype=GRB.BINARY, name="x")
m.setObjective(np.sum(A, axis=0) @ x)
m.addConstr(A @ x >= b)

m.optimize()
res = []
for v in m.getVars():
    res.append(v.x)
m.getObjective().getValue(), res

In [ ]:
x = np.array(res)

In [ ]:
A @ x >= b

In [ ]:
np.sum(A, axis=0) @ x

In [ ]:
a = [1,2,3,4,5]
b = [3,5]
a - b